### ----前置作業 下載----

In [11]:
!pip install pymssql # 連線SQL需安裝下載套件

Active code page: 936
     ---------------------------------------- 1.5/1.5 MB 3.6 MB/s eta 0:00:00


In [12]:
!pip install pyodbc # ODBC的DRIVER(驅動程式)

Active code page: 936


### ----開始執行----

In [13]:
# coding=utf-8
from platform import python_version
import os, time, socket, glob, pickle, joblib
import pandas as pd
import pyodbc, pymssql
import warnings
warnings.filterwarnings(action="ignore")
print("【現在時間】{}".format(time.strftime("%Y/%m/%d %H:%M:%S")))
print("【所在目錄】{}".format(os.getcwd()))
#print("【主機名稱】{} ({})".format(socket.gethostname(),socket.gethostbyname(socket.
print("【Python】{}".format(python_version()))
print("【pymssql】{}".format(pymssql.__version__))
print("【pyodbc】{}".format(pyodbc.version))
%autosave 120

【現在時間】2022/12/14 10:47:08
【所在目錄】C:\Users\User\Desktop\資料工程
【Python】3.9.13
【pymssql】2.2.7
【pyodbc】4.0.34


Autosaving every 120 seconds


In [1]:
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup

### ----更改股票代號----

In [54]:
stock_id = "6446" 
url = "https://goodinfo.tw/tw/StockDetail.asp?STOCK_ID=" + stock_id
payloads = {"STOCK_ID":stock_id}
headers = \
{"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}

In [6]:
url = "https://goodinfo.tw/tw/EquityDistributionClassHis.asp?STOCK_ID="+stock_id
resp = requests.request(method="POST",url=url,data=payloads,headers=headers)
resp.encoding = "utf-8"

equityDclass_table = pd.read_html(resp.text)

In [48]:
df = equityDclass_table[18].dropna(axis='columns')
df

週別   統計日期   當週股價                集保庫存(萬張) 各持股等級股東之持有比例(%)           \
        週別   統計日期     收盤  漲跌(元)   漲跌(%) 集保庫存(萬張)            ≦10張 ＞10張≦50張   
0    22W51      -    527    -10   -1.86        -               -        -   
1    22W50  12/09    537     +5   +0.94    30.27            9.91     12.9   
2    22W49  12/02    532    +26   +5.14    30.27              10     12.8   
3    22W48  11/25    506    -22   -4.17    30.27            9.93     12.5   
4    22W47  11/18    528    +20   +3.94    30.27            10.1     12.6   
..     ...    ...    ...    ...     ...      ...             ...      ...   
167  20W03  01/17    116     +1   +0.87    22.51            9.23     12.4   
168  20W02  01/10    115   +2.5   +2.22    21.94            9.48     12.6   
169  20W01  01/03  112.5  +11.5  +11.39    21.94            9.51     12.7   
170  19W52  12/27    101   +0.5    +0.5    21.94            9.58     12.7   
171  19W51  12/20  100.5   +0.6    +0.6    21.94            9.54     12.6   

                                                                
    ＞50張≦100張 ＞100張≦200張 ＞200張≦400張 ＞400張≦800張 ＞800張≦1千張  ＞1千張  
0           -          -          -          -         -     -  
1         6.7       8.04       8.25       7.08         2  45.1  
2        6.74       8.19       8.32       6.61      2.01  45.3  
3        6.48       7.84       6.94       6.41      0.84    49  
4        6.52        7.6       7.15       6.43      1.15  48.5  
..        ...        ...        ...        ...       ...   ...  
167      6.72       6.05       6.55       6.34      1.57  51.2  
168      6.91        6.1       6.81        6.5      1.65  49.9  
169      6.86       6.04       6.91       5.96      1.98    50  
170      6.68       6.25       6.73       6.04      1.99  50.1  
171      6.67       6.07       6.66       6.26      1.99  50.2  

[172 rows x 14 columns]

### ----將DataFrame 轉成list----

In [50]:
products_list = df.values.tolist()
print(products_list[1])

['22W50', '12/09', '537', '+5', '+0.94', '30.27', '9.91', '12.9', '6.7', '8.04', '8.25', '7.08', '2', '45.1']


In [59]:
'''0         1     2     3      4      5    6  7  8  9   10   11  12 '''
'''周別    日期    收盤  漲      跌  集保庫存 10 50 100 200 400 800 1000  '''
Gstock_id="GS"+str(stock_id)+str(products_list[1][0])
print(Gstock_id)

GS644622W50


In [ ]:
conn = pymssql.connect(server = '163.14.73.23', # 163.14.73.23 
 port = '1433',
 database = 'PyDB',
 user = 'team05',
 password = 'scu@team05')


In [ ]:
conn.autocommit = True
cursor = conn.cursor()
# GS+代號+發周別
# MYSQL指令
Gstock_id="GS"+str(stock_id)+str(products_list[1][0])
sqlStr = f""" 
    insert into 持股等級股東持有比例表 (持股等級股東持有比例表編號, \
    股東持有張數比例(≦10張), 股東持有張數比例(>10張&≦50張),\
    股東持有張數比例(>50張&≦100張),股東持有張數比例(＞100張≦200張),\
    股東持有張數比例(＞200張≦400張),股東持有張數比例(＞400張≦800張),\
    股東持有張數比例(＞800張≦1千張),股東持有張數比例(＞1千張),
    集保庫存(萬張),統計日期,持股等級周別,股票代號)
    values ({Gstock_id}, {products_list[1][6]}, {products_list[1][7]},{products_list[1][8]},\
    {products_list[1][9]},{products_list[1][10]},{products_list[1][11]},{products_list[1][12]},\
    {products_list[1][5]},{products_list[1][1]},{products_list[1][0]},{stock_id})
    select * from tdale
"""

cursor.execute(sqlStr)



In [ ]:
conn.close()

In [ ]:
'''
MySQL 持股等級股東持有比例表 格式

GS+代號+發周別 
比例
集保庫存
日期
周別
代號


''' 
# MySQL組合鍵
# https://www.delftstack.com/zh-tw/howto/mysql/introduction-to-the-mysql-composite-key/
